In [ ]:
!pip install --upgrade pip setuptools wheel
!apt update && apt install -y ffmpeg libcairo2-dev libpango1.0-dev
!pip install manim
!pip install -q smolagents==1.7.0
!pip install markdownify datasets sentence-transformers duckduckgo-search spaces gradio-tools langchain langchain-community langchain-huggingface faiss-cpu --upgrade -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 20.6 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 47.9 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 75.1.0
    Uninstalling setuptools-75.1.0:
      Successfully uninstalled setuptools-75.1.0
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandas-gbq 0.25.0 requires google-api-core<3.0.0dev,>=2.10.2, but you have google-api-core 1.34.1 which is incompatible.
tensorflow-decision-forests 1.10.0 requires tensorflow==2.17.0, but you have tensorflow 2.17.1 which is incompatible.
Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates I

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from smolagents import HfApiModel, CodeAgent, ManagedAgent
import subprocess
import tempfile
import os
from pathlib import Path
import shutil
import sys
from contextlib import contextmanager
import re


In [ ]:
def extract_code(response):
    """Extract code from model response"""
    code_lines = []
    in_code_block = False
    for line in response.split('\n'):
        if line.strip().startswith('```'):
            in_code_block = not in_code_block
            continue
        if in_code_block:
            code_lines.append(line)
    return '\n'.join(code_lines)


In [ ]:
class DependencyManager:
    """Manages dependencies required for Manim"""
    
    @staticmethod
    def install_dependencies():
        """Install required dependencies with specific versions"""
        dependencies = [
            "numpy",
            "scipy",
            "manim",
            "pillow",
            "pycairo",
            "manimpango",
            "networkx",
            "decorator",
            "sympy"
        ]
        
        for dep in dependencies:
            try:
                subprocess.check_call([
                    sys.executable,
                    "-m",
                    "pip",
                    "install",
                    "--no-cache-dir",
                    dep
                ])
            except subprocess.CalledProcessError as e:
                print(f"Error installing {dep}: {e}")
                return False
        return True

In [ ]:
class ManimCodeWriterAgent(ManagedAgent):
    """Agent responsible for converting educational concepts into Manim code"""
    
    def __init__(self, model):
        self.template = """
        You are an expert Manim programmer. Convert the following concept into production-ready Manim code:
        
        CONCEPT: {concept}
        
        REQUIREMENTS:
        1. Create an educational animation under 100 seconds
        2. Use ManimCE syntax
        3. Include proper documentation and comments throughout the code
        4. Use meaningful variable names and follow PEP 8 style guidelines
        5. Implement smooth transitions and animations with appropriate timing
        6. Keep the scenes visually clear and avoid cluttering the screen
        7. Use color schemes that are easy on the eyes and accessible
        8. Include a proper title and labels where appropriate
        
        CODE STRUCTURE:
        - Start with: from manim import *
        - Create ONE main Scene class that ends with 'Scene' (e.g., BinarySearchScene)
        - Break complex animations into separate methods for better readability
        - Initialize all necessary objects and configurations in the constructor
        - Handle edge cases and potential errors gracefully
        
        OPTIMIZATION GUIDELINES:
        - Use appropriate Manim methods rather than reinventing functionality
        - Leverage existing Manim objects and animations when possible
        - Minimize redundant code and hardcoded values
        - Use numpy for efficient mathematical operations
        - Cache calculations when appropriate to avoid redundant computation
        
        Return ONLY the complete, runnable Python code without any explanation.
        """
        
        super().__init__(
            agent=CodeAgent(
                tools=[],
                model=model,
                additional_authorized_imports=["manim", "numpy", "math"]
            ),
            name="manim_writer",
            description="Writes Manim code for educational animations"
        )
    
    def generate_code(self, concept):
        """Generate Manim code for the given concept"""
        prompt = self.template.format(concept=concept)
        response = self.agent.run(prompt)
        return extract_code(response)

In [ ]:
class ManimExecutorAgent(ManagedAgent):
    """Agent responsible for executing and debugging Manim code"""
    def __init__(self, model):
        self.debug_template = """
        You are an expert Manim debugger. Fix the following code that produced an error:
        
        ORIGINAL CODE:
        ```python
        {code}
        ```
        
        ERROR MESSAGE:
        ```
        {error}
        ```
        
        DEBUGGING INSTRUCTIONS:
        1. Carefully analyze the error message and identify the root cause
        2. Look for common Manim pitfalls like:
           - Incorrect object creation or parameters
           - Animation timing issues
           - Missing or invalid attributes
           - Version compatibility problems
           - Resource handling errors
        3. Make the minimum changes needed to fix the issue
        4. Preserve the original functionality and intent of the code
        5. Follow Manim best practices in your fixes
        6. Ensure the code remains compatible with ManimCE version 0.17.3
        
        Return ONLY the complete, corrected code without explanations or markdown formatting. The code should be ready to run without any modifications.
        """
        
        super().__init__(
            agent=CodeAgent(
                tools=[],
                model=model,
                additional_authorized_imports=["subprocess", "sys", "os"]
            ),
            name="manim_executor",
            description="Executes and debugs Manim code"
        )
    
    @contextmanager
    def _create_temp_dir(self):
        """Context manager for creating and cleaning up temporary directory"""
        temp_dir = tempfile.mkdtemp()
        try:
            yield temp_dir
        finally:
            if os.path.exists(temp_dir):
                shutil.rmtree(temp_dir, ignore_errors=True)
    
    def _extract_scene_name(self, code):
        """Extract the Scene class name from the code"""
        import re
        pattern = r"class\s+(\w+Scene)\s*\("
        matches = re.findall(pattern, code)
        return matches[-1] if matches else None
    
    def debug_code(self, code, error):
        """Debug the code using the model"""
        prompt = self.debug_template.format(code=code, error=error)
        response = self.agent.run(prompt)
        return extract_code(response)
    
    def execute_code(self, code):
        """Execute the Manim code and return the path to the generated video"""
        with self._create_temp_dir() as temp_dir:
            try:
                scene_name = self._extract_scene_name(code)
                if not scene_name:
                    raise Exception("Could not find a valid Scene class in the code")
                
                script_path = os.path.join(temp_dir, "scene.py")
                with open(script_path, "w") as f:
                    f.write(code)
                
                result = subprocess.run(
                    [
                        "python",
                        "-m",
                        "manim",
                        "-ql",
                        "--media_dir", temp_dir,
                        script_path,
                        scene_name
                    ],
                    capture_output=True,
                    text=True,
                    cwd=temp_dir
                )
                
                if result.returncode != 0:
                    fixed_code = self.debug_code(code, result.stderr)
                    return self.execute_code(fixed_code)
                
                videos_dir = os.path.join(temp_dir, "videos", "scene", "480p15")
                if not os.path.exists(videos_dir):
                    raise Exception(f"Videos directory not found: {videos_dir}")
                    
                video_path = os.path.join(videos_dir, f"{scene_name}.mp4")
                if not os.path.exists(video_path):
                    raise Exception(f"Video file not found: {video_path}")
                
                current_dir = os.getcwd()
                output_path = os.path.join(current_dir, f"output_{scene_name}.mp4")
                
                print(f"Copying from {video_path} to {output_path}")
                shutil.copy(video_path, output_path)
                
                if os.path.exists(output_path):
                    print(f"Successfully saved video to {output_path}")
                    return output_path
                else:
                    raise Exception(f"Failed to save video to {output_path}")
                    
            except Exception as e:
                print(f"Error during execution: {str(e)}")
                if isinstance(e, subprocess.CalledProcessError):
                    return self.debug_code(code, e.stderr)
                return self.debug_code(code, str(e))

In [ ]:
class ManimManagerAgent(ManagedAgent):
    """Manager agent that coordinates the animation creation process"""
    
    def __init__(self, model):
        if not DependencyManager.install_dependencies():
            raise RuntimeError("Failed to install required dependencies")
            
        self.writer = ManimCodeWriterAgent(model)
        self.executor = ManimExecutorAgent(model)
        
        super().__init__(
            agent=CodeAgent(
                tools=[],
                model=model,
                managed_agents=[self.writer, self.executor]
            ),
            name="manim_manager",
            description="Manages the creation of educational animations using Manim"
        )
    
    def create_animation(self, concept):
        """Coordinate the creation of a Manim animation for the given concept"""
        code = self.writer.generate_code(concept)
        video_path = self.executor.execute_code(code)
        
        return video_path

In [ ]:
model = HfApiModel("Qwen/Qwen2.5-72B-Instruct")
manager = ManimManagerAgent(model)

video_path = manager.create_animation(
    "Visualize the recursive calls in the Fibonacci sequence calculation, showing the call stack and how values are computed"
)
print(f"Animation created at: {video_path}")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ You are an expert Manim programmer. Convert the following concept into production-ready Manim code:             │
│                                                                                                                 │
│         CONCEPT: Visualize the recursive calls in the Fibonacci sequence calculation, showing the call stack    │
│ and how values are computed                                                                                     │
│                                                                                                                 │
│         REQUIREMENTS:                                                                                           │
│         1. Create an educational animation under 100 seconds                                                    │
│         2. Use ManimCE syntax                                                                                   │
│         3. Include proper documentation and comments throughout the code                                        │
│         4. Use meaningful variable names and follow PEP 8 style guidelines                                      │
│         5. Implement smooth transitions and animations with appropriate timing                                  │
│         6. Keep the scenes visually clear and avoid cluttering the screen                                       │
│         7. Use color schemes that are easy on the eyes and accessible                                           │
│         8. Include a proper title and labels where appropriate                                                  │
│                                                                                                                 │
│         CODE STRUCTURE:                                                                                         │
│         - Start with: from manim import *                                                                       │
│         - Create ONE main Scene class that ends with 'Scene' (e.g., BinarySearchScene)                          │
│         - Break complex animations into separate methods for better readability                                 │
│         - Initialize all necessary objects and configurations in the constructor                                │
│         - Handle edge cases and potential errors gracefully                                                     │
│                                                                                                                 │
│         OPTIMIZATION GUIDELINES:                                                                                │
│         - Use appropriate Manim methods rather than reinventing functionality                                   │
│         - Leverage existing Manim objects and animations when possible                                          │
│         - Minimize redundant code and hardcoded values                                                          │
│         - Use numpy for efficient mathematical operations                                                       │
│         - Cache calculations when appropriate to avoid redundant computation                                    │
│                                                                                                                 │
│         Return ONLY the complete, runnable Python code without any explanation.                                 │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-72B-Instruct ────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  from manim import *                                                                                              
                                                                                                                   
  class FibonacciRecursiveVisualizationScene(Scene):                                                               
      """Scene to visualize the recursive calls in the Fibonacci sequence calculation."""                          
                                                                                                                   
      CONFIG = {                                                                                                   
          "fibonacci_value": 6,                                                                                    
          "node_color": BLUE,                                                                                      
          "text_color": WHITE,                                                                                     
          "edge_color": GRAY,                                                                                      
          "background_color": DARK_GRAY,                                                                           
          "node_padding": 0.5,                                                                                     
          "node_radius": 0.3,                                                                                      
      }                                                                                                            
                                                                                                                   
      def construct(self):                                                                                         
          """Main method to construct the scene."""                                                                
          self.set_background()                                                                                    
          self.add_title()                                                                                         
          self.fibonacci_tree = self.create_fibonacci_tree(self.fibonacci_value)                                   
          self.animate_fibonacci_recursive(self.fibonacci_tree)                                                    
                                                                                                                   
      def set_background(self):                                                                                    
          """Set the background color of the scene."""                                                             
          self.background = Rectangle(                                                                             
              color=self.background_color,                                                                         
              fill_color=self.background_color,                                                                    
              fill_opacity=1,                                                                                      
              height=CONFIG["frame_height"],                                                                       
              width=CONFIG["frame_width"],                                                                         
          )                                                                                                        
          self.add(self.background)                                                                                
                                                                                                                   
      def add_title(self):                               

Code execution failed at line 'from manim import *' due to: AttributeError:module 'scipy.interpolate._interpolate' 
has no attribute 'RegularGridInterpolator'

[Step 0: Duration 27.10 seconds| Input tokens: 2,324 | Output tokens: 759]

...

Error in generating model output:
500 Server Error: Internal Server Error for url: 
https://api-inference.huggingface.co/models/Qwen/Qwen2.5-72B-Instruct/v1/chat/completions (Request ID: E8PoDm)

Model too busy, unable to get response in less than 60 second(s)

[Step 5: Duration 60.07 seconds| Input tokens: 43,513 | Output tokens: 4,829]

Reached max steps.

[Step 6: Duration 60.07 seconds| Input tokens: 54,274 | Output tokens: 5,598]

Copying from /tmp/tmpkgi9y6er/videos/scene/480p15/FibonacciRecursiveVisualizationScene.mp4 to /kaggle/working/output_FibonacciRecursiveVisualizationScene.mp4
Successfully saved video to /kaggle/working/output_FibonacciRecursiveVisualizationScene.mp4
Animation created at: /kaggle/working/output_FibonacciRecursiveVisualizationScene.mp4
